In [2]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()


True

In [3]:
from urllib.parse import quote_plus

# Import the credentials
username = os.getenv("USERNAME")
password = os.getenv("PASSWORD")
SERVER = os.getenv("SERVER")
DATABASE = os.getenv("DATABASE")
DRIVER = quote_plus(os.getenv("DRIVER"))

if username and password:
    connection_url = f"mssql+pyodbc://{username}:{password}@{SERVER}/{DATABASE}?driver={DRIVER}&Encrypt=no&TrustServerCertificate=yes"
else:
    connection_url = f"mssql+pyodbc://@{SERVER}/{DATABASE}?driver={DRIVER}&trusted_connection=yes"     

engine = create_engine(connection_url, fast_executemany=True)
query = "SELECT 1 as test_col"
print(pd.read_sql_query(query, con=engine))

   test_col
0         1


In [4]:
# See the tables ---> in the database
# from sqlalchemy import inspect
# inspector = inspect(engine)
# tables = inspector.get_table_names()
# print("Available Tables:", tables)


tables = pd.read_sql_query("SELECT name,create_date,modify_date FROM sys.tables", con =  engine)

# check the total count of records in each table with top 5 records
for table in tables["name"]:
    print("-" * 75,table, "-" * 75 )
    query1 = f"SELECT COUNT(*) AS count FROM {table}"
    result1 = pd.read_sql_query(query1, con = engine)["count"].values[0]
    print(f"Total Count of records for {table} :{result1}")
    query2 = f"SELECT TOP 5 * FROM {table}"
    result2 = pd.read_sql_query(query2, con = engine)
    print(result2)


--------------------------------------------------------------------------- begin_inventory ---------------------------------------------------------------------------
Total Count of records for begin_inventory :206529
         InventoryId  Store          City  Brand                  Description  \
0  1_HARDERSFIELD_58      1  HARDERSFIELD     58  Gekkeikan Black & Gold Sake   
1  1_HARDERSFIELD_60      1  HARDERSFIELD     60       Canadian Club 1858 VAP   
2  1_HARDERSFIELD_62      1  HARDERSFIELD     62     Herradura Silver Tequila   
3  1_HARDERSFIELD_63      1  HARDERSFIELD     63   Herradura Reposado Tequila   
4  1_HARDERSFIELD_72      1  HARDERSFIELD     72         No. 3 London Dry Gin   

    Size  onHand  Price   startDate  
0  750mL       8  12.99  2024-01-01  
1  750mL       7  10.99  2024-01-01  
2  750mL       6  36.99  2024-01-01  
3  750mL       3  38.99  2024-01-01  
4  750mL       6  34.99  2024-01-01  
------------------------------------------------------------------

In [5]:
# Print the columns in each table to understand the relation b/w tables
# These columns are helpful in the exploration 
for table in tables["name"]:
    print("-" * 50)
    query = f"SELECT TOP 2 * FROM {table}"
    print(f"{table}")
    df =  pd.read_sql_query(query, con = engine)
    print("-" * 50)
    print(df.columns)


--------------------------------------------------
begin_inventory
--------------------------------------------------
Index(['InventoryId', 'Store', 'City', 'Brand', 'Description', 'Size',
       'onHand', 'Price', 'startDate'],
      dtype='object')
--------------------------------------------------
end_inventory
--------------------------------------------------
Index(['InventoryId', 'Store', 'City', 'Brand', 'Description', 'Size',
       'onHand', 'Price', 'endDate'],
      dtype='object')
--------------------------------------------------
purchases
--------------------------------------------------
Index(['InventoryId', 'Store', 'Brand', 'Description', 'Size', 'VendorNumber',
       'VendorName', 'PONumber', 'PODate', 'ReceivingDate', 'InvoiceDate',
       'PayDate', 'PurchasePrice', 'Quantity', 'Dollars', 'Classification'],
      dtype='object')
--------------------------------------------------
purchase_prices
--------------------------------------------------
Index(['Brand', 'De

# About the tables

In [6]:
# Go through the table purchase to understand brands performance
ptable = "purchases"
id = 4466
query = f"SELECT * FROM {ptable} WHERE VendorNumber = {id}"
purchase = pd.read_sql_query(query , con = engine)
purchase.groupby(["Brand"])[["PurchasePrice","Quantity","Dollars"]].sum()

,PurchasePrice,Quantity,Dollars
Brand,,,
3140,7027.32,4640,51921.60
5215,6681.10,4923,46325.43
5255,7984.90,6215,58110.25


In [7]:
# Go through the table sales to understand brands performance in sales
stable = "sales"
id = 4466
query = f"SELECT * FROM {stable} WHERE VendorNo = {id}"
sales = pd.read_sql_query(query , con = engine)
sales.groupby(["Brand"])[["SalesQuantity","SalesPrice","ExciseTax","SalesDollars"]].sum()

,SalesQuantity,SalesPrice,ExciseTax,SalesDollars
Brand,,,,
3140,3890,30071.85,7149.25,50531.10
5215,4651,41542.02,8548.96,60416.49
5255,6096,51180.60,11204.28,79187.04


In [8]:
# Go through the table Purchase_prices to understand brands performance in purchases

pptable = "purchase_prices"
id = 4466
query = f"SELECT * FROM {pptable} WHERE VendorNumber = {id}"
purchase_prices = pd.read_sql_query(query , con = engine)
purchase_prices.groupby(["Brand"])[["Price","PurchasePrice"]].sum()

,Price,PurchasePrice
Brand,,
3140,14.99,11.19
5215,12.99,9.41
5255,12.99,9.35


In [9]:
# Go through the table Purchase_prices to understand brands performance in sales

vtable = "vendor_invoice"
id = 4466
query = f"SELECT * FROM {vtable} WHERE VendorNumber = {id}"
vendor_invoice = pd.read_sql_query(query, con = engine)
vendor_invoice

,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval
0,4466,AMERICAN VINTAGE BEVERAGE,2024-01-07,8137,2023-12-22,2024-02-21,15,140.55,8.57,None
1,4466,AMERICAN VINTAGE BEVERAGE,2024-01-19,8207,2023-12-27,2024-02-26,335,3142.33,16.97,None
2,4466,AMERICAN VINTAGE BEVERAGE,2024-01-18,8307,2024-01-03,2024-02-18,41,383.35,1.99,None
3,4466,AMERICAN VINTAGE BEVERAGE,2024-01-27,8469,2024-01-14,2024-03-11,72,673.20,3.30,None
4,4466,AMERICAN VINTAGE BEVERAGE,2024-02-04,8532,2024-01-19,2024-03-15,79,740.21,3.48,None
5,4466,AMERICAN VINTAGE BEVERAGE,2024-02-09,8604,2024-01-24,2024-03-15,347,3261.37,17.61,None
6,4466,AMERICAN VINTAGE BEVERAGE,2024-02-17,8793,2024-02-05,2024-04-02,72,675.36,3.17,None
7,4466,AMERICAN VINTAGE BEVERAGE,2024-03-01,8892,2024-02-12,2024-03-28,117,1096.05,5.15,None
8,4466,AMERICAN VINTAGE BEVERAGE,2024-03-07,8995,2024-02-19,2024-04-02,129,1209.27,5.44,None
9,4466,AMERICAN VINTAGE BEVERAGE,2024-03-12,9033,2024-02-22,2024-04-16,147,1377.87,6.61,None


## Aggregated table creation

In [10]:
summary1 = pd.read_sql_query("SELECT VendorNumber, SUM(Freight) as AdditionalCharges FROM vendor_invoice GROUP BY VendorNumber", con = engine)
summary1

,VendorNumber,AdditionalCharges
0,6830,360.29
1,8920,232.99
2,17032,283.61
3,7749,91.50
4,4692,9139.38
...,...,...
121,11567,20964.81
122,2450,251.28
123,7239,16978.67
124,8664,18544.49


In [11]:
summary2 = pd.read_sql_query("""
SELECT
    p.VendorNumber,
    p.VendorName,
    p.Brand,
    p.PurchasePrice,
    pp.Volume,
    pp.Price AS ActualPrice,
    SUM(p.Quantity) AS TotalQuantity,
    SUM(p.Dollars) AS TotalPurchasePrice
FROM purchases p
JOIN purchase_prices pp
    ON p.Brand = pp.Brand
WHERE p.PurchasePrice > 0
GROUP BY
    p.VendorNumber,
    p.VendorName,
    p.Brand,
    p.PurchasePrice,
    pp.Volume,
    pp.Price
ORDER BY
    TotalPurchasePrice DESC
""", con=engine)

summary2

,VendorNumber,VendorName,Brand,PurchasePrice,Volume,ActualPrice,TotalQuantity,TotalPurchasePrice
0,1128,BROWN-FORMAN CORP,1233,26.27,1750,36.99,145080,3811251.60
1,4425,MARTIGNETTI COMPANIES,3405,23.19,1750,28.99,164038,3804041.22
2,17035,PERNOD RICARD USA,8068,18.24,1750,24.99,187407,3418303.68
3,3960,DIAGEO NORTH AMERICA INC,4261,16.17,1750,22.99,201682,3261197.94
4,3960,DIAGEO NORTH AMERICA INC,3545,21.89,1750,29.99,138109,3023206.01
...,...,...,...,...,...,...,...,...
10687,9815,WINE GROUP INC,8527,1.32,750,4.99,2,2.64
10688,8004,SAZERAC CO INC,5683,0.39,50,0.49,6,2.34
10689,3924,HEAVEN HILL DISTILLERIES,9123,0.74,50,0.99,2,1.48
10690,3960,DIAGEO NORTH AMERICA INC,6127,1.47,200,1.99,1,1.47


In [12]:
summary3 = pd.read_sql_query("""SELECT
    VendorNo,
    Brand,
    SUM(SalesPrice) AS TotalSalesPrice,
    SUM(SalesQuantity) AS TotalSalesQuantity,
    SUM(ExciseTax) AS TotalExciseTax,
    SUM(SalesDollars) AS TotalSales
FROM sales s
GROUP BY VendorNo, Brand
ORDER BY TotalSales
""", con = engine)

summary3

,VendorNo,Brand,TotalSalesPrice,TotalSalesQuantity,TotalExciseTax,TotalSales
0,8004,5287,0.98,2,0.10,9.800000e-01
1,9206,2773,0.99,1,0.05,9.900000e-01
2,3252,3933,0.99,2,0.10,1.980000e+00
3,10050,3623,1.98,2,0.10,1.980000e+00
4,3924,9123,0.99,2,0.10,1.980000e+00
...,...,...,...,...,...,...
11267,3960,3545,545778.28,135838,249587.83,4.223108e+06
11268,3960,4261,420050.01,200412,368242.80,4.475973e+06
11269,17035,8068,461140.15,187140,343854.07,4.538121e+06
11270,4425,3405,561512.37,160247,294438.66,4.819073e+06


#### Create agg table via joins 

### For me it took more than 15 mintues so i would not recommend for anyone to run this code

In [13]:
# import time 
# st = time.time()
# summary4 = pd.read_sql_query("""
# SELECT
#     pp.VendorNumber,
#     pp.Brand,
#     pp.Price AS ActualPrice,
#     pp.PurchasePrice,
#     SUM(s.SalesQuantity) AS TotalQuantity,
#     SUM(s.SalesDollars) AS TotalSalesDollars,
#     SUM(s.SalesPrice) AS TotalSalesPrice,
#     SUM(s.ExciseTax) AS TotalExciseTax,
#     SUM(vi.Quantity) AS TotalPurchaseQuantity,
#     SUM(vi.Dollars) AS TotalPurchaseDollars,
#     SUM(vi.Freight) AS TotalFreightCost                        
# FROM purchase_prices pp
# JOIN Sales s
#     ON pp.VendorNumber = s.VendorNo AND pp.Brand = s.Brand
# JOIN vendor_invoice vi
#     ON pp.VendorNumber = vi.VendorNumber 
# GROUP BY
#     pp.VendorNumber,
#     pp.Brand,
#     pp.Price,
#     pp.PurchasePrice
# """, con=engine)

# end = time.time() 
# print(f'Total Time : {end - st} seconds')
# summary4.head()




##### **Performance Optimization**:

- The above commented query uses joins after the aggregations is performed since we have very dataset we need to optimise the query
- Storing the aggregated results before joining the data helps in avoiding the heavy computations.


In [17]:
vendor_summary = pd.read_sql_query("""
WITH FreightSummary AS (
    SELECT 
        VendorNumber, 
        SUM(Freight) AS TotalAdditionalCharges 
    FROM vendor_invoice 
    GROUP BY VendorNumber
),
PurchaseSummary AS (
    SELECT
        p.VendorNumber,
        p.VendorName,
        p.Description,
        p.Brand,
        p.PurchasePrice,
        pp.Price AS ActualPrice,
        pp.Volume,
        SUM(p.Quantity) AS TotalPurchaseQuantity,
        SUM(p.Dollars) AS TotalPurchaseDollars
    FROM purchases p
    JOIN purchase_prices pp
        ON p.Brand = pp.Brand
    WHERE p.PurchasePrice > 0
    GROUP BY
        p.VendorNumber,
        p.VendorName,
        p.Description,
        p.Brand,
        p.PurchasePrice,
        pp.Price,
        pp.Volume
),
SalesSummary AS (
    SELECT
        VendorNo,
        Brand,
        SUM(SalesQuantity) AS TotalSalesQuantity,
        SUM(SalesDollars) AS TotalSalesDollars,
        SUM(ExciseTax) AS TotalExciseTax,
        SUM(SalesPrice) AS TotalSalesPrice
    FROM sales s
    GROUP BY VendorNo, Brand
)
-- Main Query
SELECT
    ps.VendorNumber,
    ps.VendorName,
    ps.Description,
    ps.Brand,
    ps.PurchasePrice,
    ps.ActualPrice,
    ps.Volume,
    ps.TotalPurchaseQuantity,
    ps.TotalPurchaseDollars,
    ss.TotalSalesQuantity,
    ss.TotalSalesDollars,
    ss.TotalExciseTax,
    fs.TotalAdditionalCharges,
    ss.TotalSalesPrice
FROM PurchaseSummary ps
LEFT JOIN SalesSummary ss
    ON ps.VendorNumber = ss.VendorNo AND ps.Brand = ss.Brand
LEFT JOIN FreightSummary fs
    ON ps.VendorNumber = fs.VendorNumber
ORDER BY ps.TotalPurchaseDollars DESC
""", con=engine)
vendor_summary


,VendorNumber,VendorName,Description,Brand,PurchasePrice,ActualPrice,Volume,TotalPurchaseQuantity,TotalPurchaseDollars,TotalSalesQuantity,TotalSalesDollars,TotalExciseTax,TotalAdditionalCharges,TotalSalesPrice
0,1128,BROWN-FORMAN CORP,Jack Daniels No 7 Black,1233,26.27,36.99,1750,145080,3811251.60,142049.0,5.101920e+06,260999.20,68601.68,672819.31
1,4425,MARTIGNETTI COMPANIES,Tito's Handmade Vodka,3405,23.19,28.99,1750,164038,3804041.22,160247.0,4.819073e+06,294438.66,144929.24,561512.37
2,17035,PERNOD RICARD USA,Absolut 80 Proof,8068,18.24,24.99,1750,187407,3418303.68,187140.0,4.538121e+06,343854.07,123780.22,461140.15
3,3960,DIAGEO NORTH AMERICA INC,Capt Morgan Spiced Rum,4261,16.17,22.99,1750,201682,3261197.94,200412.0,4.475973e+06,368242.80,257032.07,420050.01
4,3960,DIAGEO NORTH AMERICA INC,Ketel One Vodka,3545,21.89,29.99,1750,138109,3023206.01,135838.0,4.223108e+06,249587.83,257032.07,545778.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,9815,WINE GROUP INC,Concannon Glen Ellen Wh Zin,8527,1.32,4.99,750,2,2.64,5.0,1.595000e+01,0.55,27100.41,10.96
10688,8004,SAZERAC CO INC,Dr McGillicuddy's Apple Pie,5683,0.39,0.49,50,6,2.34,134.0,6.566000e+01,7.04,50293.62,1.47
10689,3924,HEAVEN HILL DISTILLERIES,Deep Eddy Vodka,9123,0.74,0.99,50,2,1.48,2.0,1.980000e+00,0.10,14069.87,0.99
10690,3960,DIAGEO NORTH AMERICA INC,The Club Strawbry Margarita,6127,1.47,1.99,200,1,1.47,72.0,1.432800e+02,15.12,257032.07,77.61


##### **Aggregated Table Transformations**

In [ ]:
# Understand the data in the aggregated table and Check the datatypes of each column.
vendor_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10692 entries, 0 to 10691
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   VendorNumber            10692 non-null  int64  
 1   VendorName              10692 non-null  object 
 2   Description             10692 non-null  object 
 3   Brand                   10692 non-null  int64  
 4   PurchasePrice           10692 non-null  float64
 5   ActualPrice             10692 non-null  float64
 6   Volume                  10692 non-null  object 
 7   TotalPurchaseQuantity   10692 non-null  int64  
 8   TotalPurchaseDollars    10692 non-null  float64
 9   TotalSalesQuantity      10514 non-null  float64
 10  TotalSalesDollars       10514 non-null  float64
 11  TotalExciseTax          10514 non-null  float64
 12  TotalAdditionalCharges  10692 non-null  float64
 13  TotalSalesPrice         10514 non-null  float64
dtypes: float64(8), int64(3), object(3)
mem

In [19]:
vendor_summary.head()

,VendorNumber,VendorName,Description,Brand,PurchasePrice,ActualPrice,Volume,TotalPurchaseQuantity,TotalPurchaseDollars,TotalSalesQuantity,TotalSalesDollars,TotalExciseTax,TotalAdditionalCharges,TotalSalesPrice
0,1128,BROWN-FORMAN CORP,Jack Daniels No 7 Black,1233,26.27,36.99,1750,145080,3811251.60,142049.0,5.101920e+06,260999.20,68601.68,672819.31
1,4425,MARTIGNETTI COMPANIES,Tito's Handmade Vodka,3405,23.19,28.99,1750,164038,3804041.22,160247.0,4.819073e+06,294438.66,144929.24,561512.37
2,17035,PERNOD RICARD USA,Absolut 80 Proof,8068,18.24,24.99,1750,187407,3418303.68,187140.0,4.538121e+06,343854.07,123780.22,461140.15
3,3960,DIAGEO NORTH AMERICA INC,Capt Morgan Spiced Rum,4261,16.17,22.99,1750,201682,3261197.94,200412.0,4.475973e+06,368242.80,257032.07,420050.01
4,3960,DIAGEO NORTH AMERICA INC,Ketel One Vodka,3545,21.89,29.99,1750,138109,3023206.01,135838.0,4.223108e+06,249587.83,257032.07,545778.28


In [20]:
# change the datatype of volume as it is stored as object but values are numbers
vendor_summary["Volume"] = vendor_summary["Volume"].astype("float64")

In [22]:
# Perform some basic cleaning checks 
# if missing values > 0 where remove or fill missing values if any exist
if vendor_summary.isna().sum().any():
    vendor_summary.fillna(0, inplace=True)


In [23]:
# Perform Duplicates check 
# if duplicates values > 0 where remove those duplicates as it may affect analysis
if vendor_summary.duplicated().sum() > 0:
    vendor_summary.drop_duplicates()

In [24]:
# Peform Inconsistency checks --> 
# Leading, Trailing or irregular spacing for categorical columns
for col in vendor_summary.select_dtypes(exclude="number").columns:
    print(f"Unique values in column '{col}' before stripping:")
    print(vendor_summary[col].unique())
    
    # Strip leading/trailing spaces and replace multiple spaces with single space
    vendor_summary[col] = vendor_summary[col].str.strip().str.replace(r'\s+', ' ', regex=True)
    
    print(f"\nUnique values in column '{col}' after stripping:")
    print(vendor_summary[col].unique())
    print("-" * 50)

Unique values in column 'VendorName' before stripping:
['BROWN-FORMAN CORP          ' 'MARTIGNETTI COMPANIES'
 'PERNOD RICARD USA          ' 'DIAGEO NORTH AMERICA INC   '
 'BACARDI USA INC            ' 'JIM BEAM BRANDS COMPANY    '
 'MAJESTIC FINE WINES        ' 'ULTRA BEVERAGE COMPANY LLP '
 'STOLI GROUP,(USA) LLC      ' 'PROXIMO SPIRITS INC.       '
 'MOET HENNESSY USA INC      ' 'CAMPARI AMERICA            '
 'SAZERAC CO INC             ' 'CONSTELLATION BRANDS INC   '
 'M S WALKER INC             ' 'SAZERAC NORTH AMERICA INC. '
 'PALM BAY INTERNATIONAL INC ' 'REMY COINTREAU USA INC     '
 'SIDNEY FRANK IMPORTING CO  ' 'E & J GALLO WINERY         '
 'WILLIAM GRANT & SONS INC   ' 'HEAVEN HILL DISTILLERIES   '
 'DISARONNO INTERNATIONAL LLC' 'EDRINGTON AMERICAS         '
 'CASTLE BRANDS CORP.        ' 'SOUTHERN WINE & SPIRITS NE '
 'STE MICHELLE WINE ESTATES  ' 'TRINCHERO FAMILY ESTATES   '
 'MHW LTD                    ' 'WINE GROUP INC             '
 'PERFECTA WINES             ' 'LUXC

## **Data Enrichment**:
##### Add new columns for additional information that might be useful for further analysis

- GrossProfit : Amount of profit after selling a product i.e purchase cost - selling cost
- ProfitMargin : The profit percentage 
- StockTurnOver : The total no.of quantity sold for purchased quantity
 SalesToPurchaseRatio : The ratio of sales to purchase

In [25]:
vendor_summary["GrossProfit"] = vendor_summary["TotalSalesDollars"] - vendor_summary["TotalPurchaseDollars"]
vendor_summary["ProfitMargin"] = (vendor_summary["GrossProfit"] /  vendor_summary["TotalSalesDollars"] ) * 100
vendor_summary["StockTurnOver"] = vendor_summary["TotalSalesQuantity"] / vendor_summary["TotalPurchaseQuantity"]
vendor_summary["SalesToPurchaseRatio"] = vendor_summary["TotalSalesDollars"] / vendor_summary["TotalPurchaseDollars"]

In [26]:
vendor_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10692 entries, 0 to 10691
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   VendorNumber            10692 non-null  int64  
 1   VendorName              10692 non-null  object 
 2   Description             10692 non-null  object 
 3   Brand                   10692 non-null  int64  
 4   PurchasePrice           10692 non-null  float64
 5   ActualPrice             10692 non-null  float64
 6   Volume                  10692 non-null  float64
 7   TotalPurchaseQuantity   10692 non-null  int64  
 8   TotalPurchaseDollars    10692 non-null  float64
 9   TotalSalesQuantity      10692 non-null  float64
 10  TotalSalesDollars       10692 non-null  float64
 11  TotalExciseTax          10692 non-null  float64
 12  TotalAdditionalCharges  10692 non-null  float64
 13  TotalSalesPrice         10692 non-null  float64
 14  GrossProfit             10692 non-null

In [27]:
vendor_summary.head()

,VendorNumber,VendorName,Description,Brand,PurchasePrice,ActualPrice,Volume,TotalPurchaseQuantity,TotalPurchaseDollars,TotalSalesQuantity,TotalSalesDollars,TotalExciseTax,TotalAdditionalCharges,TotalSalesPrice,GrossProfit,ProfitMargin,StockTurnOver,SalesToPurchaseRatio
0,1128,BROWN-FORMAN CORP,Jack Daniels No 7 Black,1233,26.27,36.99,1750.0,145080,3811251.60,142049.0,5.101920e+06,260999.20,68601.68,672819.31,1290667.91,25.297693,0.979108,1.338647
1,4425,MARTIGNETTI COMPANIES,Tito's Handmade Vodka,3405,23.19,28.99,1750.0,164038,3804041.22,160247.0,4.819073e+06,294438.66,144929.24,561512.37,1015032.27,21.062810,0.976890,1.266830
2,17035,PERNOD RICARD USA,Absolut 80 Proof,8068,18.24,24.99,1750.0,187407,3418303.68,187140.0,4.538121e+06,343854.07,123780.22,461140.15,1119816.92,24.675786,0.998575,1.327594
3,3960,DIAGEO NORTH AMERICA INC,Capt Morgan Spiced Rum,4261,16.17,22.99,1750.0,201682,3261197.94,200412.0,4.475973e+06,368242.80,257032.07,420050.01,1214774.94,27.139908,0.993703,1.372493
4,3960,DIAGEO NORTH AMERICA INC,Ketel One Vodka,3545,21.89,29.99,1750.0,138109,3023206.01,135838.0,4.223108e+06,249587.83,257032.07,545778.28,1199901.61,28.412764,0.983556,1.396897


##### **Storing the Aggregated Table in Database will** :
- Helps in analyzing the sales, purchases and pricing for different vendors and brands easily
- Instead of running expensive queries everytime, dashboard can fetch data quickly from aggregated table

##### Now move the aggTable to the SQL Server database